### Translate fasta containing amino acid sequences to comma-free code:

In [5]:
%load_ext blackcellmagic
from Bio import SeqIO

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [6]:
# comme-free code AA dictionary
cfcode = {
    "F": "ACC",
    "L": "ACA",
    "I": "ATA",
    "M": "ATC",
    "V": "ATT",
    "S": "CTA",
    "P": "CTC",
    "T": "CTT",
    "A": "AGA",
    "Y": "AGC",
    "H": "AGT",
    "Q": "AGG",
    "N": "CGA", 
    "K": "CGC",
    "D": "CGT",
    "E": "CGG",
    "C": "TGA",
    "W": "TGC",
    "R": "TGT",
    "G": "TGG",
    "X": "NNN",  # Amino acid not known
    "B": "CGT",  # Represents either N or D - will translate as D here (N is only off by one base)
    "J": "ACA",  # Represents either L or I - will translate as L here (I is only off by one base)
    "Z": "CGG"   # Represents either E or Q - will translate as E here (Q is only off by one base)
}

In [7]:
fasta = "../../Pachter_lab/comma-free/palmdb/2021-03-14/uniques.fa"

ids = []
seqs = []
cfc_seqs = []
seq_lens = []
for record in SeqIO.parse(fasta, "fasta"):
    # Translate AA sequence to comma free
    cfc_seq_temp = []
    for aa in record.seq:
        cfc_seq_temp.append(cfcode[aa])
    
    # Record seq
    seqs.append(record.seq)
    # Record sequence length
    seq_lens.append(len(record.seq))

    # Store cfc sequences and IDs in list
    cfc_seqs.append("".join(cfc_seq_temp))
    ids.append(record.id)

In [8]:
print(f"Number of sequences: {len(ids)}")
print(f"AA sequence lengths (min-max): {min(seq_lens)} - {max(seq_lens)}")
# Check if all IDs are unique
print("IDs are unique: ",len(ids) == len(set(ids)))
# Check if all sequences are unique
print("Sequences are unique: ", len(cfc_seqs) == len(set(cfc_seqs)))
print("Number of shared sequences: ", len(cfc_seqs) - len(set(cfc_seqs)))

Number of sequences: 296623
AA sequence lengths (min-max): 50 - 250
IDs are unique:  True
Sequences are unique:  False
Number of shared sequences:  62


Hamming distances between sequences in virus reference:

In [ ]:
%%time
from scipy.spatial.distance import hamming
import itertools

# Get Hamming distances in AA sequences
hamming_distances = []
for seq1, seq2 in itertools.combinations(cfc_seqs, 2):
     # Compute Hamming distance with scipy package (returns percentage that can be converted to Hamming distance by multiplying by length of array)
    if len(seq1) > len(seq2):
        hamming_distance = hamming(list(seq1[:len(seq2)]), list(seq2)) * len(list(seq2))
    elif len(seq2) > len(seq1):
        hamming_distance = hamming(list(seq1), list(seq2[:len(seq1)])) * len(list(seq1))
    else:
        hamming_distance = hamming(list(seq1), list(seq2)) * len(list(seq1))
    hamming_distances.append(hamming_distance)

# Get Hamming distances in cfc sequences
hamming_distances_cfc = []
for seq1, seq2 in itertools.combinations(cfc_seqs, 2):
    # Compute Hamming distance with scipy package (returns percentage that can be converted to Hamming distance by multiplying by length of array)
    if len(seq1) > len(seq2):
        hamming_distance = hamming(list(seq1[:len(seq2)]), list(seq2)) * len(list(seq2))
    elif len(seq2) > len(seq1):
        hamming_distance = hamming(list(seq1), list(seq2[:len(seq1)])) * len(list(seq1))
    else:
        hamming_distance = hamming(list(seq1), list(seq2)) * len(list(seq1))
    hamming_distances_cfc.append(hamming_distance)
    
# Plot Hamming distances    
fig, axs = plt.subplots(figsize=(10,5), ncols=2)

ax=axs[0]
ax.hist(hamming_distances, 5)
ax.set_title("Hamming distances in PalmDB reference")
ax.set_xlabel("Hamming distance")
ax.set_ylabel("Frequency")

ax=axs[1]
ax.hist(hamming_distances_cfc, 5)
ax.set_title("Hamming distances in PalmDB reference\n(after translation to comma-free code)")
ax.set_xlabel("Hamming distance")
ax.set_ylabel("Frequency")

fig.show()

In [ ]:
fig.savefig("palmdb_hdists.png", dpi=300, bbox_inches="tight")

### Build dna and gtf files

In [2]:
path_to_folder = "../../Pachter_lab/comma-free/palmdb/2021-03-14"

In [ ]:
with open(f"{path_to_folder}/cfc_palmdb_annotation.gtf", "w") as gtf, open(
    f"{path_to_folder}/cfc_palmdb_genome.fa", "w"
) as dna:
    genome_name = "CFCpalmdb1"

    # Add header lines to GTF
    gtf.write(
        f"#!genome-build {genome_name}.1\n#!genome-version {genome_name}\n#!genome-date 2021-03-14\n#!genome-build-accession {genome_name}\n#!genebuild-last-updated 2021-03-14\n")

    start = 1
    chromosome = 1
    for cfc_seq, id in zip(cfc_seqs, ids):
        source = "palmdb"
        features = ["gene", "transcript", "exon", "CDS"]
        frames = [".", ".", ".", "0"]
        end = start  + len(cfc_seq)
        gene_id = id

        for feature, frame in zip(features, frames):
            if feature == "gene":
                gtf.write(
                    f'{chromosome}\t{source}\t{feature}\t{start} {end} .\t+\t{frame}\tgene_id "{gene_id}"; gene_version "1"; gene_name "{gene_id}"; gene_source "palmdb"; gene_biotype "protein_coding";\n'
                    )
            if feature == "transcript":
                gtf.write(
                    f'{chromosome}\t{source}\t{feature}\t{start} {end} .\t+\t{frame}\tgene_id "{gene_id}"; gene_version "1"; transcript_id "{gene_id}T"; transcript_version "1"; gene_name "{gene_id}"; gene_source "palmdb"; gene_biotype "protein_coding"; transcript_name "{gene_id}"; transcript_source "palmdb"; transcript_biotype "protein_coding"; tag "CCDS"; ccds_id "CCDS{gene_id}"; tag "basic";\n'
                )
            if feature == "exon":
                gtf.write(
                    f'{chromosome}\t{source}\t{feature}\t{start} {end} .\t+\t{frame}\tgene_id "{gene_id}"; gene_version "1"; transcript_id "{gene_id}T"; transcript_version "1"; exon_number "1"; gene_name "{gene_id}"; gene_source "palmdb"; gene_biotype "protein_coding"; transcript_name "{gene_id}"; transcript_source "palmdb"; transcript_biotype "protein_coding"; tag "CCDS"; ccds_id "CCDS{gene_id}"; exon_id "{gene_id}E"; exon_version "1"; tag "basic";\n'
                )
            if feature == "CDS":
                gtf.write(
                    f'{chromosome}\t{source}\t{feature}\t{start} {end} .\t+\t{frame}\tgene_id "{gene_id}"; gene_version "1"; transcript_id "{gene_id}T"; transcript_version "1"; exon_number "1"; gene_name "{gene_id}"; gene_source "palmdb"; gene_biotype "protein_coding"; transcript_name "{gene_id}"; transcript_source "palmdb"; transcript_biotype "protein_coding"; tag "CCDS"; ccds_id "CCDS{gene_id}"; protein_id "{gene_id}P"; protein_version "1"; tag "basic";\n'
                    )

        # Build dna/genome file
        dna.write(f">{chromosome} dna:chromosome chromosome:{genome_name}:{chromosome}:{start}:{end}:1 REF\n")
        dna.write(f"{cfc_seq}\n")

        # Add next PALM sequence to new chromosome
        chromosome += 1

Create t2g:

In [ ]:
!cat uniques.fa | awk '{if($1~">") print $1"\t"$1}' > cfc_palmdb_t2g.txt
!sed -i'.original' 's/>u//g' cfc_palmdb_t2g.txt
!rm cfc_palmdb_t2g.txt.original

## Build index and bus file

In [ ]:
# # Generate index with kb (DOES NOT WORK - DELANEY SAYS HE HAS THE SAME PROBLEM WITH KB REF)
# !kb ref \
#     -i $path_to_folder/kb/cfc_palmdb_kb_index.idx \
#     -g $path_to_folder/kb/cfc_palmdb_t2g.txt \
#     -f1 $path_to_folder/kb/cfc_palmdb_transcriptome.fa \
#     $path_to_folder/cfc_palmdb_genome.fa \
#     $path_to_folder/cfc_palmdb_annotation.gtf

In [ ]:
# Generate kallisto index
!kallisto index \
    -i $path_to_folder/cfc_palmdb_index.idx \
    $path_to_folder/cfc_palmdb_genome.fa


[build] loading fasta file ../../../Pachter_lab/comma-free/palmdb/2021-03-14/cfc_palmdb_genome.fa
[build] k-mer length: 31
[build] warning: replaced 114207 non-ACGUT characters in the input sequence
        with pseudorandom nucleotides
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 38457212
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 8030027
CompactedDBG::filter(): Processed 88645809 k-mers in 296623 reads
CompactedDBG::filter(): Found 38301304 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 262893
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting uniti